<a href="https://colab.research.google.com/github/fokams/COMPRESSION-D-UN-FICHIER-EN-UTILISANT-LE-CODAGE-DE-HUFFMAN/blob/main/ProgGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook
This notebook is a starting point to setup for CUDA code.
If you want to save your codes, you will have to create a copy of this notebook, and then work on this copy

# CUDA installation
Start by enabling GPU acceleration in your notebook : go to Modify->Notebook settings and select GPU as hardware accelerator.

Then, check if your instance has CUDA installed :



In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# Prepare your notebook to handle CUDA code

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s1s69iv9
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s1s69iv9
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7432 sha256=ad99cf1e6175f37f13398802e9e82d42885745e385f9d1c2ce678f3d359f3a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-577z5xu_/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmpwjn64q1x".


# Install & Use of CCfits for TD 2


In [ ]:
!apt-get install libccfits-dev

In [ ]:
import astropy.io.fits as pf
import numpy as np

A = np.random.random((10,10))
pf.writeto("test.fits", A, overwrite=True)

In [ ]:
%%writefile test2.cu
#include <stdio.h>
#include <CCfits>

int main(void) {
    CCfits::FITS inFile("test.fits", CCfits::Read, true);
    CCfits::PHDU &phdu = inFile.pHDU();
    std::valarray<float> fitsImage;
    phdu.read(fitsImage);
    std::cout << fitsImage[1] << std::endl;
    std::cout << fitsImage.size() << std::endl;
    long naxes[2] = {10,10};
    CCfits::FITS *inFile2 = new CCfits::FITS("test2.fits", FLOAT_IMG, 2, naxes);
    CCfits::PHDU &phdu2 = inFile2->pHDU();
    phdu2.write(1, 99, fitsImage);
}


Overwriting test2.cu


In [ ]:
!nvcc test2.cu -o test2 -I/usr/include/CCfits -L/usr/lib/x86_64-linux-gnu/ -std=c++11 -lCCfits -lcfitsio

In [ ]:
!./test2

In [ ]:
import astropy.io.fits as pf
import numpy as np
A = pf.getdata("test2.fits")
B = pf.getdata("test.fits")

# You are ready to code in CUDA

In [ ]:
%%cuda
#include <iostream>
__global__ void kernel(void){
}

int main(void){
    kernel<<<1,1>>>();
    printf("Hello World");
    return 0;
}

Hello World


In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git

Cloning into 'cuda-samples'...
remote: Enumerating objects: 12369, done.
remote: Counting objects: 100% (12369/12369), done.
remote: Compressing objects: 100% (1882/1882), done.
remote: Total 12369 (delta 10491), reused 12320 (delta 10461), pack-reused 0
Receiving objects: 100% (12369/12369), 130.29 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (10491/10491), done.
Checking out files: 100% (3660/3660), done.


In [ ]:
!cd Samples/1_Utilities/deviceQuery ; make SMS="75"

/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common  -m64    --threads 0 --std=c++11 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_37,code=compute_37 -o deviceQuery.o -c deviceQuery.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_37,code=sm_37 -gencode arch=compute_37,code=compute_37 -o deviceQuery deviceQuery.o 
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mkdir -p ../../../bin/x86_64/linux/release
cp deviceQuery ../../../bin/x86_64/linux/release


In [ ]:
!cd Samples/1_Utilities/deviceQuery ; ./deviceQuery

./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          11.2 / 11.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15110 MBytes (15843721216 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shar